In [1]:
!pip install pandarallel
import pandas as pd
from bs4 import BeautifulSoup
import requests
import glob
import numpy as np
from pandarallel import pandarallel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pandarallel: filename=pandarallel-1.6.1-py3-none-any.whl size=16265 sha256=47618e33d920e4753b76cafbf7d63e8d619aec5baab411186fbaf28932510848
  Stored in directory: /root/.cache/pip/wheels/e8/81/8f/29aafd7d671a4e9db4e976d71728870173f0b3f48ccba0bc32
Successfully built pandarallel


In [56]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [57]:
df=pd.DataFrame()
read_file = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/eldiariootin/url.csv', header=[0])
df=pd.concat([df,read_file]).reset_index(drop=True)
print(len(df))

5886


In [58]:
def get_articles(url):
  try:
    r1 = requests.get(url)
    coverpage = r1.content
    soup = BeautifulSoup(coverpage, 'html.parser')
    try:
      coverpage= soup.find_all('h1',class_='tdb-title-text')
      title=coverpage[0].get_text()
      # print(title)
    except:
      title=''

    try:
      coverpage= soup.find_all('time',class_='entry-date updated td-module-date')
      time=coverpage[0]['datetime']
      # print(title)
    except:
      time=''

    try:
      coverpage = soup.find_all('div', class_='vc_column-inner')
      article=coverpage[5].find('div',class_='tdb-block-inner td-fix-index').get_text()
      # for i in coverpage[0].find_all('p'):
      #   article=article+i.get_text()
      # print(article)
    except:
      article=''
      print('error\n'+url)
  except:
    title=''
    description=''
    article=''
    print('url error\n'+url)    

  return [title,time,article]

In [55]:
get_articles('https://www.elmostrador.cl/noticias/mundo/2017/07/12/hacia-donde-se-dirige-el-iceberg-que-se-acaba-de-separar-de-la-antartica/')

error
https://www.elmostrador.cl/noticias/mundo/2017/07/12/hacia-donde-se-dirige-el-iceberg-que-se-acaba-de-separar-de-la-antartica/


['', '', '']

In [59]:
pandarallel.initialize(nb_workers= 50, progress_bar=True)

output=df['rows'].parallel_apply(get_articles)

df['country']='colombia'
df['title']=''
df['time']=''
df['article']=''

for i in range(0,len(df)):
  df.loc[i]['title','time','article']=output[i]

df = df[df.title.str.len()!=0].reset_index(drop=True)
df = df.dropna()
str_name='/content/drive/MyDrive/Colab Notebooks/eldiariootin/url_article.csv'
print(df)
df.to_csv(str_name, index=False)

INFO: Pandarallel will run on 50 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


error
https://www.eldiario.com.co/categoria/judicial/
error
https://www.eldiario.com.co/judicial/507172/
error
https://www.eldiario.com.co/judicial/438726/
error
https://www.eldiario.com.co/categoria/noticias/politica/
error
https://www.eldiario.com.co/judicial/495081/
error
https://www.eldiario.com.co/judicial/435257/
error
https://www.eldiario.com.co/judicial/523630/
                                                   page  \
0     https://www.eldiario.com.co/categoria/judicial...   
1     https://www.eldiario.com.co/categoria/judicial...   
2     https://www.eldiario.com.co/categoria/judicial...   
3     https://www.eldiario.com.co/categoria/judicial...   
4     https://www.eldiario.com.co/categoria/judicial...   
...                                                 ...   
5872  https://www.eldiario.com.co/categoria/noticias...   
5873  https://www.eldiario.com.co/categoria/noticias...   
5874  https://www.eldiario.com.co/categoria/noticias...   
5875  https://www.eldiario.com.co/cate